## Import Required Classes ##

In [31]:
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
from keras.preprocessing import image

## Data collection ##

In [32]:
#Image dimensions:
img_width, img_height = 150, 150

train_data_dir = 'data/image-processing/train'
validation_data_dir = 'data/image-processing/test'

#Clissification
nb_train_samples = 1000
nb_validation_samples = 500 #800
epochs = 50 #loop counter to send batch for analysis 
batch_size = 20 #16 #1 batch will have 20 images

if backend.image_data_format() == 'channels_first': 
    #RGB first then width and height
    input_shape = (3, img_width, img_height)
else:
    #150,150,3 # Width at first then height and RGB
    input_shape = (img_width, img_height, 3)

#Generate Training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#this is the augmentation configuration we will use for testing: only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Nuralnetwork Creation ##

In [33]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=input_shape))
#Conv2D: Convinutional network used to extract feature from the image. Here 32 distinct features will be extracted within 3X3 pixels
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.summary()
print('-------------------------------------------------------------------------------------------------------------------')

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

print('-------------------------------------------------------------------------------------------------------------------')
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu')) # relu: returns either 0 or 1
model.add(MaxPooling2D(pool_size=(2,2)))

print('-------------------------------------------------------------------------------------------------------------------')
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_22 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 74, 74, 32)        0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________
-------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------
Model: "sequential_6"
________________

## Generate Data for Above Nural Network ##

In [35]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size)
model.save_weights('first_try.h5')

Epoch 1/50
50/50 [==============================] - 32s 642ms/step - loss: 0.4198 - accuracy: 0.8090 - val_loss: 0.6482 - val_accuracy: 0.6920
Epoch 2/50
50/50 [==============================] - 47s 953ms/step - loss: 0.3557 - accuracy: 0.8650 - val_loss: 0.4757 - val_accuracy: 0.7900
Epoch 3/50
50/50 [==============================] - 47s 932ms/step - loss: 0.3123 - accuracy: 0.8650 - val_loss: 0.8613 - val_accuracy: 0.7280
Epoch 4/50
50/50 [==============================] - 38s 755ms/step - loss: 0.2789 - accuracy: 0.9010 - val_loss: 0.7089 - val_accuracy: 0.7640
Epoch 5/50
50/50 [==============================] - 37s 745ms/step - loss: 0.3073 - accuracy: 0.8800 - val_loss: 0.3699 - val_accuracy: 0.8240
Epoch 6/50
50/50 [==============================] - 40s 799ms/step - loss: 0.2719 - accuracy: 0.8916 - val_loss: 0.5985 - val_accuracy: 0.8120
Epoch 7/50
50/50 [==============================] - 38s 762ms/step - loss: 0.2912 - accuracy: 0.8880 - val_loss: 0.3647 - val_accuracy: 0.8480

## PREDECTION ##

In [37]:
img_pred = image.load_img('data/image-processing/val/NORMAL/NORMAL2-IM-1431-0001.jpeg', target_size=(150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis = 0)

rslt = model.predict(img_pred)
print(rslt)
if rslt[0][0] == 1:
    prediction = "Normal"
else:
    prediction = 'Infected'
print(prediction)

[[1.]]
Normal
